# Continuous Delivery for Helm Charts on Kubernetes Engine using Concourse

This tutorial shows how to create a software release process with Kubernetes Engine, Helm, and Concourse. Helm is a tool to help you manage Kubernetes manifests. Concourse takes advantage of Helm to continuously deploy your applications.

Following this tutorial results in:

Two source code repositories, one for your application source code and another for your Helm chart.
Your application, packaged as a Docker container and installed and configured as a Helm Chart in your cluster.
You can push a Git tag to either of the two repositories to start a release.

## Architecture


Concourse assembles continuous delivery pipelines that you can use to codify the steps of your build, test, and release processes. In Concourse, the stages of pipelines are called jobs. Each job can take a resource as an input and can create a resource as an output.

In this tutorial, you use the following Concourse resource types:

Docker images
Git repositories and tags
Helm charts and releases
You can use Helm to make your Kubernetes manifests into templates and then configure, install, and upgrade them as as a unit. You must define your application as a chart for Helm to install it. Each Helm chart has a values file that you can use to parameterize your manifests. For example, you might have a value that defines the Docker image to use for your application deployment. This value might change each time you install or upgrade your chart. Each installation of a chart is called a release. You use releases to upgrade or roll back an instance of your application.

You can share charts by using a repository. In this tutorial, you create a private chart repository using Cloud Storage.

## Objectives

## Before you begin


## Setting up your environment


### Create a Kubernetes Engine cluster


In [0]:
gcloud container clusters create concourse --image-type ubuntu \
    --machine-type n1-standard-2 --zone us-central1-f \
    --scopes cloud-source-repos-ro,storage-full

### Download the sample code


In [0]:
wget https://gke-concourse.storage.googleapis.com/sample-app-v4.zip
unzip sample-app-v4.zip
cd concourse-continuous-delivery-master

## Deploying Concourse using Helm


### Install Helm


In [0]:
wget https://storage.googleapis.com/kubernetes-helm/helm-v2.6.2-linux-amd64.tar.gz

In [0]:
tar zxfv helm-v2.6.2-linux-amd64.tar.gz
cp linux-amd64/helm .

In [0]:
kubectl create clusterrolebinding user-admin-binding --clusterrole=cluster-admin --user=$(gcloud config get-value account)

In [0]:
kubectl create serviceaccount tiller --namespace kube-system

In [0]:
kubectl create clusterrolebinding tiller-admin-binding --clusterrole=cluster-admin --serviceaccount=kube-system:tiller

In [0]:
kubectl create clusterrolebinding --clusterrole=cluster-admin --serviceaccount=default:default concourse-admin

In [0]:
./helm init --service-account=tiller
./helm update

In [0]:
export PROJECT=$(gcloud info --format='value(config.project)')
export BUCKET=$PROJECT-helm-repo
./helm plugin install https://github.com/viglesiasce/helm-gcs.git --version v0.1.1
gsutil mb -l us-central1 gs://$BUCKET
./helm gcs init gs://$BUCKET

In [0]:
./helm version

In [0]:
Client: &version.Version{SemVer:"v2.6.2",
GitCommit:"012cb0ac1a1b2f888144ef5a67b8dab6c2d45be6",
GitTreeState:"clean"}Server: &version.Version{SemVer:"v2.6.2",
GitCommit:"012cb0ac1a1b2f888144ef5a67b8dab6c2d45be6",
GitTreeState:"clean"}

### Deploy Concourse


In [0]:
export PASSWORD=$(openssl rand -base64 15)
cat > concourse.yaml <<EOF
concourse:
  password: $PASSWORD
  baggageclaimDriver: overlay
web:
  service:
    type: LoadBalancer
EOF

In [0]:
./helm install stable/concourse --name concourse -f concourse.yaml --version 0.10.0

In [0]:
kubectl get pods -l app=concourse-web


In [0]:
export SERVICE_IP=$(kubectl get svc \
    --namespace default concourse-web \
    -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
wget -O fly "http://$SERVICE_IP:8080/api/v1/cli?arch=amd64&platform=linux"
chmod +x fly

In [0]:
./fly -t local login -u concourse -p $PASSWORD -c http://$SERVICE_IP:8080


In [0]:
export SERVICE_IP=$(kubectl get svc \
    --namespace default concourse-web \
    -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
printf "Concourse URL: [http://$SERVICE_IP:8080]\nUsername: concourse\nPassword: $PASSWORD\n"

### Configure identity and access management


In [0]:
gcloud iam service-accounts create concourse --display-name concourse

In [0]:
export SA_EMAIL=$(gcloud iam service-accounts list \
    --filter="displayName:concourse" --format='value(email)')
export PROJECT=$(gcloud info --format='value(config.project)')

In [0]:
gcloud projects add-iam-policy-binding $PROJECT \
    --role roles/storage.admin --member serviceAccount:$SA_EMAIL

In [0]:
gcloud iam service-accounts keys create concourse-sa.json \
    --iam-account $SA_EMAIL

## Deploying your application


### Create source code repositories


In [0]:
gcloud source repos create chart-source
gcloud source repos create app-source

In [0]:
git config --global user.email "[EMAIL_ADDRESS]"
git config --global user.name "[USERNAME]"

In [0]:
export PROJECT=$(gcloud info --format='value(config.project)')
for repo in app-source chart-source; do
cd $repo
git init && git add . && git commit -m 'Initial commit'
git config credential.helper gcloud.sh
git remote add google \
    https://source.developers.google.com/p/$PROJECT/r/$repo
    git push --all google
cd ..
done

### Configure and create the pipeline


In [0]:
export PROJECT=$(gcloud info --format='value(config.project)')
export BUCKET=$PROJECT-helm-repo
export TOKEN_SECRET=$(kubectl get serviceaccount default -o jsonpath="{.secrets[0].name}")
export CLUSTER_CA=$(kubectl get secret $TOKEN_SECRET -o jsonpath='{.data.ca\.crt}')
export TOKEN=$(kubectl get secret $TOKEN_SECRET -o jsonpath='{.data.token}' | base64 --decode)

cat > params.yaml <<EOF
chart_name: nginx
release_name: dev-site
bucket: $BUCKET
cluster_ca: $CLUSTER_CA
token: $TOKEN
project: $PROJECT
service_account_json: '$(cat concourse-sa.json)'
EOF

In [0]:
./fly -t local set-pipeline -p dev-site-deploy \
    -c pipeline.yaml -l params.yaml -n

In [0]:
./fly -t local unpause-pipeline -p dev-site-deploy

### Deploying your application for the first time


In [0]:
for repo in app-source chart-source; do
    cd $repo
    git tag v1.0.0
    git push google --tags
    cd ..
done

### Deploying a change to the application


In [0]:
export POD_NAME=$(kubectl get pods --namespace default \
    -l "app=nginx,release=dev-site" \
    -o jsonpath="{.items[0].metadata.name}")
kubectl port-forward $POD_NAME 8080:80 &
curl -is localhost:8080 | grep 'Server\|color'

In [0]:
killall kubectl

In [0]:
cd app-source

In [0]:
sed -i s/stable/latest/ Dockerfile

In [0]:
git add Dockerfile
git commit -m 'Use latest NGINX'
git tag v2.0.0

In [0]:
git push google --mirror

In [0]:
../fly -t local check-resource -r dev-site-deploy/app-source

In [0]:
export POD_NAME=$(kubectl get pods --namespace default \
    -l "app=nginx,release=dev-site" \
    -o jsonpath="{.items[0].metadata.name}")
kubectl port-forward $POD_NAME 8080:80 &
curl -is localhost:8080 | grep 'Server\|color'

In [0]:
export POD_NAME=$(kubectl get pods --namespace default \
    -l "app=nginx,release=dev-site" \
    -o jsonpath="{.items[0].metadata.name}")
kubectl port-forward $POD_NAME 8080:80 &
curl -is localhost:8080 | grep 'Server\|color'

In [0]:
Server: nginx/1.15.0
<h1 style="color:blue;">Welcome to the sample app!</h1>

In [0]:
killall kubectl

### Deploying a change to the chart


In [0]:
cd ../chart-source/

In [0]:
sed -i s/blue/green/ templates/config-map.yaml

In [0]:
git add templates/config-map.yaml
git commit -m 'Use green for page heading'

In [0]:
git tag v2.0.0
git push google --mirror

In [0]:
../fly -t local check-resource -r dev-site-deploy/chart-source

In [0]:
export POD_NAME=$(kubectl get pods \
    -n default -l "app=nginx,release=dev-site" \
    -o jsonpath="{.items[0].metadata.name}")
kubectl port-forward $POD_NAME 8080:80 &
curl -is localhost:8080 | grep 'Server\|color'

## Cleaning up


In [0]:
../helm delete --purge concourse

In [0]:
export SA_EMAIL=$(gcloud iam service-accounts list \
    --filter="displayName:concourse" \
    --format='value(email)')
gcloud iam service-accounts delete $SA_EMAIL

In [0]:
gcloud container clusters delete concourse --zone us-central1-f

In [0]:
gcloud source repos delete app-source --quiet
gcloud source repos delete chart-source --quiet

In [0]:
export PROJECT=$(gcloud info --format='value(config.project)')
export BUCKET=$PROJECT-helm-repo
gsutil -m rm -r gs://$BUCKET